In [29]:
import pandas as pd
import numpy as np
import sqlite3
from convokit import Corpus

In [30]:
SUBREDDIT = "technology"
PARENT_DIRECTORY = "./datasets/" + SUBREDDIT + "/"

CONVERSATIONS_PER_SUBREDDIT = 20
UTTERANCES_PER_CONVERSATION = 10

'''
Inclusive for both
Sum each time:

(For START_PRIMARY_KEY_CONVERSATION_TABLE)
CONVERSATIONS_PER_SUBREDDIT

(For START_PRIMARY_KEY_UTTERANCE_TABLE)
CONVERSATIONS_PER_SUBREDDIT * (UTTERANCES_PER_CONVERSATION + STARTER_UTTERANCE)
'''

# Sumar 20 y 220

START_PRIMARY_KEY_CONVERSATION_TABLE = 120
START_PRIMARY_KEY_UTTERANCE_TABLE = 1320

# Generate database

In [31]:
%%sh

SUBREDDIT=technology
PARENT_DIRECTORY=./datasets/${SUBREDDIT}/

grep -P '"reply_to": ?null' ${PARENT_DIRECTORY}utterances.jsonl > ${PARENT_DIRECTORY}starter_utterances.jsonl

# Remove [deleted]
tmpFile="$(mktemp)"
grep -Pv '"text": ?"\[deleted\]"' ${PARENT_DIRECTORY}starter_utterances.jsonl > "$tmpFile"
cat "$tmpFile" > ${PARENT_DIRECTORY}starter_utterances.jsonl
rm "$tmpFile"

# Remove [removed]
tmpFile="$(mktemp)"
grep -Pv '"text": ?"\[removed\]"' ${PARENT_DIRECTORY}starter_utterances.jsonl > "$tmpFile"
cat "$tmpFile" > ${PARENT_DIRECTORY}starter_utterances.jsonl
rm "$tmpFile"

# Remove links
tmpFile="$(mktemp)"
grep -Pv '"text": ?"[^"]*?https?://' ${PARENT_DIRECTORY}starter_utterances.jsonl > "$tmpFile"
cat "$tmpFile" > ${PARENT_DIRECTORY}starter_utterances.jsonl
rm "$tmpFile"

# Remove text with 250 characters or more
tmpFile="$(mktemp)"
grep -Pv '"text": ?"(\\"|[^"]){250}' ${PARENT_DIRECTORY}starter_utterances.jsonl > "$tmpFile"
cat "$tmpFile" > ${PARENT_DIRECTORY}starter_utterances.jsonl
rm "$tmpFile"

# Remove text with 100 characters or less
tmpFile="$(mktemp)"
grep -Pv '"text": ?"(\\"|[^"]){0,100}+"' ${PARENT_DIRECTORY}starter_utterances.jsonl > "$tmpFile"
cat "$tmpFile" > ${PARENT_DIRECTORY}starter_utterances.jsonl
rm "$tmpFile"

echo '"conversation_id"' > ${PARENT_DIRECTORY}conversation_ids.csv
grep -Po '(?<="root": )"[^"]+"' ${PARENT_DIRECTORY}starter_utterances.jsonl >> ${PARENT_DIRECTORY}conversation_ids.csv

In [34]:
conversation_ids = pd.read_csv(PARENT_DIRECTORY + "conversation_ids.csv")

In [35]:
conversation_ids["conversation_id"]

0       ckxoc
1       ckxtw
2       cl20s
3       cl32j
4       co85o
        ...  
3240    fblkq
3241    fcndt
3242    fcfwx
3243    fccck
3244    fc73j
Name: conversation_id, Length: 3245, dtype: object

In [36]:
conversations_df = pd.read_json(PARENT_DIRECTORY + "conversations.json", orient = "index")

In [37]:
conversations_df.loc[:, :] = conversations_df[conversations_df.index.isin(conversation_ids["conversation_id"])]

/tmp/ipykernel_3427/1488128701.py:1: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  conversations_df.loc[:, :] = conversations_df[conversations_df.index.isin(conversation_ids["conversation_id"])]


In [38]:
conversations_df = conversations_df.dropna(how = "all")

In [39]:
conversations_df

,title,num_comments,domain,timestamp,subreddit,gilded,gildings,stickied,author_flair_text
ckxoc,What computer should I buy?,1.0,self.technology,2010-07-01 13:59:02,technology,0.0,None,False,
ckxtw,Which is the best LED TV out there for me to u...,10.0,self.technology,2010-07-01 14:13:20,technology,0.0,None,False,
cl20s,Question for devs: is it possible to make a ch...,0.0,self.technology,2010-07-01 19:51:25,technology,0.0,None,False,
cl32j,Whats a good way to show a layman the differen...,11.0,self.technology,2010-07-01 21:20:39,technology,0.0,None,False,
co85o,Can someone explain to me what 3g is.,5.0,self.technology,2010-07-11 03:57:28,technology,0.0,None,False,
...,...,...,...,...,...,...,...,...,...
fblkq,"Friend has a Galaxy S, if anyone has one could...",4.0,self.technology,2011-01-30 01:46:49,technology,-1.0,None,False,
fcndt,Where did the embedded Youtube full-screen but...,0.0,self.technology,2011-01-31 21:55:40,technology,-1.0,None,False,
fcfwx,"My mom, receiving a new Win7 laptop as a gift:...",0.0,self.technology,2011-01-31 16:14:20,technology,-1.0,None,False,
fccck,HTC Freestyle Deals,0.0,self.technology,2011-01-31 12:22:58,technology,-1.0,None,False,


In [40]:
round(conversations_df.isnull().sum()/len(conversations_df.index) * 100, 2)

title                 0.00
num_comments          0.00
domain                0.00
timestamp             0.00
subreddit             0.00
gilded                0.00
gildings             99.94
stickied              0.00
author_flair_text     0.00
dtype: float64

In [41]:
conversations_df.isnull().sum()

title                   0
num_comments            0
domain                  0
timestamp               0
subreddit               0
gilded                  0
gildings             3243
stickied                0
author_flair_text       0
dtype: int64

In [42]:
# Remove the column gildings because it contains almost all its values with NaN
conversations_df = conversations_df.drop(columns=["gildings"])

In [43]:
conversations_df["timestamp"] = pd.to_datetime(conversations_df["timestamp"])

In [44]:
# Remove conversations that contains a video or a clip
conversations_df = conversations_df[~conversations_df["title"].str.contains("\[(Video|Clip)\]")]

/tmp/ipykernel_3427/1334158877.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  conversations_df = conversations_df[~conversations_df["title"].str.contains("\[(Video|Clip)\]")]


In [45]:
# Title should contain "?" to be a question
conversations_df = conversations_df[conversations_df["title"].str.contains('\?')]

In [46]:
# Select the conversations with most comments
conversations_df = conversations_df.sort_values(by=["num_comments", "timestamp"], ascending=False).head(CONVERSATIONS_PER_SUBREDDIT)

In [47]:
conversation_ids_most_comments = conversations_df.index

In [48]:
conversation_ids_most_comments

Index(['4v65u2', 'b44gh', '4yvzpa', 'aq5mo', 'bjebs', '3fdhep', '3ls6yw',
       '40dl6z', 'g75n8', '5k1iy2', '4qzs3i', '2qi0z3', 'a3dz4', 'aj21p',
       '9hgst', '9wsph', '3nbgpg', 'd0wg5', '9kmmb', '2lyo2p'],
      dtype='object')

In [49]:
# Create a file to be accessed from the bash script
with open(PARENT_DIRECTORY + "conversation_ids_most_comments.txt", "w") as file:
    file.write(str(conversation_ids_most_comments))

Shell script to filter utterances with conversation ids with most comments

In [51]:
%%sh

SUBREDDIT=technology
PARENT_DIRECTORY=./datasets/${SUBREDDIT}/

conversation_ids_most_comments_formatted="$(cat ${PARENT_DIRECTORY}conversation_ids_most_comments.txt | perl -p -e 's/\n//g' | grep -Po '(?<=\[)[^]]+' | perl -p -e "s/( |')//g" | perl -p -e 's/,/|/g' | perl -p -e 's/^(.)/(\1/g' | perl -p -e 's/(.)$/\1)/g')"
grep -P "\"root\": ?\"$conversation_ids_most_comments_formatted\"" ${PARENT_DIRECTORY}utterances.jsonl > ${PARENT_DIRECTORY}utterances_most_comments.jsonl

In [52]:
corpus = Corpus(filename = PARENT_DIRECTORY + "utterances_most_comments.jsonl")
utterances_df = corpus.get_utterances_dataframe()

In [53]:
round(utterances_df.isnull().sum()/len(utterances_df) * 100, 2)

timestamp                   0.00
text                        0.00
speaker                     0.00
reply_to                    0.35
conversation_id             0.00
meta.score                  0.00
meta.top_level_comment      0.35
meta.retrieved_on           0.00
meta.gilded                 0.00
meta.gildings             100.00
meta.subreddit              0.00
meta.stickied               0.00
meta.permalink              0.00
meta.author_flair_text      0.00
vectors                     0.00
dtype: float64

In [54]:
utterances_df.isnull().sum()

timestamp                    0
text                         0
speaker                      0
reply_to                    20
conversation_id              0
meta.score                   0
meta.top_level_comment      20
meta.retrieved_on            0
meta.gilded                  0
meta.gildings             5661
meta.subreddit               0
meta.stickied                0
meta.permalink               0
meta.author_flair_text       0
vectors                      0
dtype: int64

In [55]:
# Column meta.gildings contains almost all its values with NaN
utterances_df = utterances_df.drop(columns=["meta.gildings"])

In [ ]:
if "[deleted]" in utterances_df["speaker"].value_counts().index:
    no_names = utterances_df["speaker"].value_counts().loc["[deleted]"] + len(utterances_df["speaker"].unique()) - 1
else:
    no_names = len(utterances_df["speaker"].unique())

In [75]:
names_df = pd.read_csv("./datasets/names.csv")

200

In [100]:
names_sample_df = names_df.sample(n = no_names)

In [109]:
names_sample_df["name"]

14242             Brook
44308             Jevin
13965          Briseida
91468             Tenor
36375             Henri
              ...      
83409           Shailyn
37852              Inba
63234    Mariechristine
27375            Eardie
21322            Damato
Name: name, Length: 3315, dtype: object

In [110]:
for current, new in zip(utterances_df["speaker"].unique(), names_sample_df["name"]):
    if current == "[deleted]":
        
    else:
        utterances_df.loc[utterances_df["speaker"] == current, "speaker"] = new

sandeepail Brook
JudDredd Jevin
[deleted] Briseida
MWizzle Tenor
indite Henri
Zeratas Geroldine
Bitcoin_Chief Deneil
MonkAang Sonder
thirdprojectaccount Sirenna
thematrix307 Shanniya
thecapitalc Ahsen
RedDyeNumber4 Hilbert
yalogin Heshy
jpatton89 Tyniah
MisterKpak Rodrik
juliusseizure Mazekeen
bustduster Qualani
Hiroler Remiah
Captain_Harlock Nodin
illektr1k Chiquia
vacationingfool Aadhira
MechaBlue Goff
anarchistica Armina
homophone_police Vikie
marikas Kingman
dhca89 Barney
carefulclaire Daeshia
darkkish Brandt
jonp5065 Anadia
spacetoast Rekisha
Sunoiki Cathren
cassildacastaigne Petronia
jamesinc Miriam
gsw8 Charizma
ltjboy03 Jahmall
fuckbuddy Harlene
MassesOfTheOpiate Rawland
arcticfox Maxim
mindhacker Leviah
MuuaadDib Tersea
ParsonsProject93 Lus
noveltyicecream Briggitte
dorkasaurus Italia
RobbStark Graysonn
candyman420 Mtisha
timeshifter_ Idhika
DrunkenTrom Marliese
cassidoodle Felizardo
silence_hr Leeshaun
btgeekboy Lolah
twister6284 Benjahmin
RampantAI Taheerah
mrthomsen Rosane


In [106]:
utterances_df["speaker"].unique()

array(['sandeepail', 'JudDredd', '[deleted]', ..., 'mpez0', 'ottolite',
       'Anon_Logic'], dtype=object)

In [107]:
utterances_df.loc[utterances_df["speaker"] == "sandeepail", "speaker"]

id
4yvzpa     sandeepail
d6qsovq    sandeepail
d6qtwds    sandeepail
d6qtz5t    sandeepail
d6qu0ht    sandeepail
d6quqbi    sandeepail
d6quykh    sandeepail
d6qvj2h    sandeepail
d6qvp15    sandeepail
d6qvzx9    sandeepail
d6qwgr5    sandeepail
d6qwk3f    sandeepail
d6qxukd    sandeepail
d6qyorz    sandeepail
d6r014g    sandeepail
Name: speaker, dtype: object

In [246]:
# Create another dataframe to preserve starter utterances
starter_utterances_df = utterances_df[utterances_df["reply_to"].isnull()]
utterances_df = utterances_df[utterances_df["reply_to"].notnull()]

In [247]:
# Remove rows that at least one element is equal to "[deleted]" or to "[removed]"
utterances_df = utterances_df[~(utterances_df.eq("[deleted]").any(axis="columns")) & ~(utterances_df.eq("[removed]").any(axis="columns"))]

In [248]:
utterances_df = utterances_df[~utterances_df["text"].str.contains("https?://")]

In [249]:
utterances_df = utterances_df[(utterances_df["text"].str.len() > 100) & (utterances_df["text"].str.len() < 200)]

In [250]:
unique_conversation_ids = utterances_df["conversation_id"].unique()

In [251]:
unique_conversation_ids

array(['orw9i', 'xy6tb', '10er03', '17d832', '1buewi', '1gra3d', '1hhbnk',
       '1qzbjh', '1z04b6', '3a0znq', '3i37jg', '3nsgh6', '3yserm',
       '5gzall', '6zegb9', '7izns8', '81tw7o', '8a8gzg', '8rw63b',
       '8zf5u1'], dtype=object)

In [252]:
# Remove nested utterances
for conversation_id in unique_conversation_ids:
    # Top level utterances with most score
    utterances_df.loc[utterances_df["conversation_id"] == conversation_id, :] = utterances_df[(utterances_df["conversation_id"] == conversation_id) & (utterances_df["reply_to"] == conversation_id)].sort_values(by=["meta.score", "timestamp"], ascending=False).head(UTTERANCES_PER_CONVERSATION)

In [253]:
# Put starter utterances to the original dataframe
utterances_df = pd.concat([starter_utterances_df, utterances_df])

In [254]:
# There are a lot of rows with NaN values because I left only top level utterances
utterances_df = utterances_df.dropna(how = "all")

In [255]:
# Column "vectors" cannot be saved in database 
utterances_df = utterances_df.drop(columns=["vectors"])

In [256]:
# Tests to check dataframes

STARTER_UTTERANCE = 1

if len(utterances_df.index) == CONVERSATIONS_PER_SUBREDDIT * (UTTERANCES_PER_CONVERSATION + STARTER_UTTERANCE):
    if len(utterances_df[utterances_df["reply_to"].isnull()].index) == CONVERSATIONS_PER_SUBREDDIT:
        print("utterances_df is correct")
    else:
        print("utterances_df is incorrect")

if len(conversations_df.index) == CONVERSATIONS_PER_SUBREDDIT:
    print("conversations_df is correct")
else:
    print("conversations_df is incorrect")

utterances_df is correct
conversations_df is correct


In [257]:
# Change foreign keys of utterances dataframe
current_foreign_keys = conversations_df.index
new_foreign_keys = np.arange(START_PRIMARY_KEY_CONVERSATION_TABLE, START_PRIMARY_KEY_CONVERSATION_TABLE + len(conversations_df.index))

for current_foreign_key, new_foreign_key in zip(current_foreign_keys, new_foreign_keys):
    utterances_df.loc[utterances_df["conversation_id"] == current_foreign_key, "conversation_id"] = new_foreign_key 

In [258]:
# Change primary keys of conversations dataframe
new_primary_keys = new_foreign_keys

conversations_df.index = new_primary_keys

In [259]:
# Change primary keys of utterances dataframe
new_primary_keys = np.arange(START_PRIMARY_KEY_UTTERANCE_TABLE, START_PRIMARY_KEY_UTTERANCE_TABLE + len(utterances_df.index))

utterances_df.index = new_primary_keys

In [260]:
# Convert types of the columns of utterances_df

types = {
    "timestamp": "int64",
    "conversation_id": "int64",
    "meta.score": "int64",
    "meta.retrieved_on": "int64",
    "meta.gilded": "int64",
    "meta.stickied": "bool"
    }

utterances_df = utterances_df.astype(types)

In [261]:
# Change "." by "-" in utterances_df
names = {
    "meta.score": "meta_score", 
    "meta.top_level_comment": "meta_top_level_comment",
    "meta.retrieved_on": "meta_retrieved_on",
    "meta.gilded": "meta_gilded",
    "meta.subreddit": "meta_subreddit",
    "meta.stickied": "meta_stickied",
    "meta.permalink": "meta_permalink",
    "meta.author_flair_text": "meta_author_flair_text"
    }

utterances_df = utterances_df.rename(columns=names)

In [262]:
# Convert "timestamp" column to Unix timestamp
conversations_df["timestamp"] = (conversations_df["timestamp"] - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

In [263]:
# Remove trailing and leading spaces

utterances_df["text"] = utterances_df["text"].str.strip("\t\r\n ")

conversations_df["title"] = conversations_df["title"].str.strip("\t\r\n ")

In [264]:
database = PARENT_DIRECTORY + "subreddit_" + SUBREDDIT + ".db"
conn = sqlite3.connect(database)

In [265]:
conversations_df.to_sql(name="conversation_table", con=conn, index_label = "conversation_id")
utterances_df.to_sql(name="utterance_table", con=conn, index_label = "utterance_id")
conn.close()

# Analyze database

In [39]:
database = PARENT_DIRECTORY + "subreddit_" + SUBREDDIT + ".db"
conn = sqlite3.connect(database)

In [40]:
conversations_df = pd.read_sql_query("SELECT * FROM conversation_table", conn, index_col = "conversation_id")
utterances_df = pd.read_sql_query("SELECT * FROM utterance_table", conn, index_col = "utterance_id")

conn.commit()
conn.close()

In [41]:
conversations_df

,title,num_comments,domain,timestamp,subreddit,gilded,stickied,author_flair_text
conversation_id,,,,,,,,
0,What anime do you regret watching?,2226,self.anime,1358520593,anime,0,0,
1,Revel in shame: You Most Embarrassing Weebo Mo...,2121,self.anime,1464797230,anime,0,0,http://myanimelist.net/animelist/piphan
2,What Anime have you rated a 10/10?,1917,self.anime,1461229784,anime,0,0,http://myanimelist.net/animelist/DoctorWhoops
3,What are your actual top 3 anime? Don't give u...,1659,self.anime,1459184585,anime,0,0,
4,What has been your lowest/most weeb moment ever?,1624,self.anime,1437498357,anime,0,0,http://myanimelist.net/animelist/nosoy
5,What is an anime you dropped despite people sa...,1578,self.anime,1499322836,anime,0,0,
6,At what point did you realize you are no longe...,1566,self.anime,1521035905,anime,0,0,
7,What's an opinion you're pretty sure only you ...,1554,self.anime,1451329819,anime,0,0,
8,"What anime that is considered as ""trash"" by th...",1507,self.anime,1520863904,anime,0,0,https://myanimelist.net/profile/QuazzyWazzy


In [42]:
utterances_df

,timestamp,text,speaker,reply_to,conversation_id,meta_score,meta_top_level_comment,meta_retrieved_on,meta_gilded,meta_subreddit,meta_stickied,meta_permalink,meta_author_flair_text
utterance_id,,,,,,,,,,,,,
0,1461229784,"Title pretty much says it all, what are your 1...",DoctorWhoops,None,2,1022,None,1463562565,0,anime,0,/r/anime/comments/4fs5z9/what_anime_have_you_r...,http://myanimelist.net/animelist/DoctorWhoops
1,1459184585,We're all guilty of lying at some point. Pleas...,ParachuteMonkey,None,3,442,None,1463503182,0,anime,0,/r/anime/comments/4cb1ct/what_are_your_actual_...,
2,1451329819,I know that we get a lot of unpopular opinion ...,jamsterbuggy,None,7,184,None,1454958121,0,anime,0,/r/anime/comments/3yk0pd/whats_an_opinion_your...,
3,1405352561,"We all have different opinions on anime, so wh...",Isaacmo,None,17,123,None,1441268813,0,anime,0,/r/anime/comments/2aoaar/what_anime_do_you_jus...,
4,1464797230,First thing that comes to mind:\n\nIt was 6th ...,Piph,None,1,1812,None,1472735793,0,anime,0,/r/anime/comments/4m201p/revel_in_shame_you_mo...,http://myanimelist.net/animelist/piphan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
215,1531151498,"We've had a fair few anime based on D&amp;D, b...",Beckymetal,8xcnap,9,755,e22cijw,1535836743,0,anime,0,/r/anime/comments/8xcnap/what_concept_are_you_...,https://anilist.co/user/Beckymetal/animelist
216,1531152086,An anime about pro wrestling fully aware that ...,TheRisingTide,8xcnap,9,229,e22d911,1535837100,0,anime,0,/r/anime/comments/8xcnap/what_concept_are_you_...,
217,1531154366,I still want a shounen battle series about bre...,BeefiousMaximus,8xcnap,9,251,e22g3o0,1535838437,0,anime,0,/r/anime/comments/8xcnap/what_concept_are_you_...,


# Merge several databases

In [266]:
%load_ext sql
%sql sqlite:///reddit.db

In [267]:
%%sql

CREATE TABLE conversation_table (
    conversation_id INTEGER PRIMARY KEY NOT NULL,
    title TEXT NOT NULL,
    num_comments INTEGER NOT NULL,
    domain TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    subreddit TEXT NOT NULL,
    gilded INTEGER NOT NULL,
    stickied INTEGER NOT NULL,
    author_flair_text TEXT NOT NULL
);

CREATE TABLE utterance_table (
    utterance_id INTEGER PRIMARY KEY NOT NULL,
    timestamp INTEGER NOT NULL,
    text TEXT NOT NULL,
    speaker TEXT NOT NULL,
    reply_to TEXT,
    conversation_id INTEGER NOT NULL,
    meta_score INTEGER NOT NULL,
    meta_top_level_comment TEXT,
    meta_retrieved_on INTEGER NOT NULL,
    meta_gilded INTEGER NOT NULL,
    meta_subreddit TEXT NOT NULL,
    meta_stickied INTEGER NOT NULL,
    meta_permalink TEXT NOT NULL,
    meta_author_flair_text TEXT NOT NULL
);

ATTACH DATABASE '/home/orlando/reddit_databases/anime/subreddit_anime.db' AS anime;
ATTACH DATABASE '/home/orlando/reddit_databases/conspiracy/subreddit_conspiracy.db' AS conspiracy;
ATTACH DATABASE '/home/orlando/reddit_databases/relationships/subreddit_relationships.db' AS relationships;

BEGIN TRANSACTION;
    INSERT INTO main.conversation_table SELECT * FROM anime.conversation_table;
    INSERT INTO main.utterance_table SELECT * FROM anime.utterance_table;

    INSERT INTO main.conversation_table SELECT * FROM conspiracy.conversation_table;
    INSERT INTO main.utterance_table SELECT * FROM conspiracy.utterance_table;
    
    INSERT INTO main.conversation_table SELECT * FROM relationships.conversation_table;
    INSERT INTO main.utterance_table SELECT * FROM relationships.utterance_table;

 * sqlite:///reddit.db
Done.
Done.
Done.
Done.
Done.
Done.
20 rows affected.
220 rows affected.
20 rows affected.
220 rows affected.
20 rows affected.
220 rows affected.


[]